In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from transform_features_nn import (
    extract, scrub_activity, scrub_text_change, concatenate_essay_from_logs
)

2023-11-19 19:32:38.669237: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 19:32:38.700701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 19:32:38.700723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 19:32:38.701656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 19:32:38.706659: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 19:32:38.707081: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
PATH_TRAIN_LOGS = "./data/external/train_logs.csv"

In [3]:
X_train_logs = extract(PATH_TRAIN_LOGS)
X_train_logs = scrub_activity(X_train_logs)
X_train_logs = scrub_text_change(X_train_logs)

X_train_logs = [x for _, x in X_train_logs.groupby('id')]
essays_text = pd.concat(
    [concatenate_essay_from_logs(x) for x in X_train_logs],
    axis=0
)
# keras TextVectorization does not recognize emdash as punctuation
essays_text['essay'] = essays_text['essay'].str.replace("—", " ") 

y = pd.read_csv("./data/external/train_scores.csv")
y.rename(columns={'score': 'y'}, inplace=True)
XY = pd.merge(essays_text, y, how='left')
X, y = XY['essay'].to_numpy(), XY['y'].to_numpy()
X, X_test, y, y_test = train_test_split(X, y, test_size=0.33, random_state=777)

In [4]:
BATCH_SIZE = 32

# in tf Dataset structure, one element is one X-y pair 
XY_train = tf.data.Dataset.from_tensor_slices((X, y)).batch(BATCH_SIZE)
X_train = XY_train.map(lambda x, y: x)

XY_test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)
X_test = XY_test.map(lambda x, y: x)

In [5]:
text_vectorization = tf.keras.layers.TextVectorization(
    # with anonymized text, downscale recommended vocabulary size by magnitude 
    max_tokens=20000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=2,
    output_mode='tf_idf'
    )

text_vectorization.adapt(X_train)
# values = text_vectorization.get_vocabulary()

tfidf_XY_train = XY_train.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)

tfidf_XY_test = XY_test.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)

In [6]:
n_tokens = text_vectorization.vocabulary_size()
n_tokens

397

In [7]:
values = text_vectorization.get_vocabulary()

In [11]:
inputs = keras.Input(shape=(n_tokens,))
x = keras.layers.Dense(64, activation="relu")(inputs)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# model = keras.Sequential([
#     keras.layers.Dense(16, activation='relu'),
#     keras.layers.Dense(1)
#     ])

model.compile(
    optimizer="rmsprop",
    loss="mean_squared_error"
)
# model.summary()

In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(
    tfidf_XY_train.cache(),
    validation_data=tfidf_XY_test.cache(),
    epochs=100,
    callbacks=[callback]
    )

Epoch 1/100
52/52 [==============================] - 0s 3ms/step - loss: 12.6763 - val_loss: 1.5945
Epoch 2/100
52/52 [==============================] - 0s 1ms/step - loss: 2.7766 - val_loss: 1.3970
Epoch 3/100
52/52 [==============================] - 0s 1ms/step - loss: 2.7011 - val_loss: 1.3196
Epoch 4/100
52/52 [==============================] - 0s 1ms/step - loss: 2.3632 - val_loss: 1.2533
Epoch 5/100
52/52 [==============================] - 0s 1ms/step - loss: 2.3903 - val_loss: 1.2649
Epoch 6/100
52/52 [==============================] - 0s 1ms/step - loss: 2.1846 - val_loss: 1.4092
Epoch 7/100
52/52 [==============================] - 0s 1ms/step - loss: 1.9886 - val_loss: 1.2284
Epoch 8/100
52/52 [==============================] - 0s 1ms/step - loss: 2.0336 - val_loss: 1.1141
Epoch 9/100
52/52 [==============================] - 0s 1ms/step - loss: 1.8659 - val_loss: 1.0622
Epoch 10/100
52/52 [==============================] - 0s 1ms/step - loss: 1.8926 - val_loss: 1.0969
Epoch 11

In [13]:
# ngram=4:
    # with punctuation: validation mse min is 0.6
    # same without punctuation

# ngram=2
    # losing punctuation: again, mse ~0.6
    # with punctuation: mse worsens
